# Exporting the Notebook to Other Formats

One of the benefits of working within the notebook format is that the notebook can be converted to many other types of formats. This is beneficial for a number of reasons:

1. If we want to start prototyping in the notebook, but eventually move to a Python script, we can export the notebook to Python rather than having to copy and paste.
2. If we want to create a static version of the notebook (e.g., to email to colleagues), we can export the notebook to HTML or PDF (though, full disclosure, exporting to PDF can sometimes be finicky---so I prefer exporting to HTML).
3. If we want to report on our findings via a blog post, we can export the notebook to Markdown or RST.

Many export formats can be accessed through the notebook interface itself, by choosing **File$\rightarrow$Download As** in the menu. We can also export to different formats by using the `jupyter nbconvert` command line tool. Using the command line tool also gives us more flexibility in terms of allowing us to specify different options and templates.

---

## Converting to Python

In general, the way to use nbconvert is:

```bash
jupyter nbconvert --to [file format] [name of notebook]
```

So, for example, to export to python:

In [ ]:
%%bash

jupyter nbconvert --to python "1 - Reproducible Plots with Matplotlib.ipynb"

We can now look at the resulting file here: [1 - Reproducible Plots with Matplotlib.py](1 - Reproducible Plots with Matplotlib.py)

---

## Converting to Standalone HTML

Similarly, to convert to HTML we use `--to html`. We can also pass the `--execute` flag to have nbconvert first execute the notebook before converting it to HTML:

In [ ]:
%%bash

jupyter nbconvert --execute --to html "1 - Reproducible Plots with Matplotlib.ipynb"

[1 - Reproducible Plots with Matplotlib.html](1 - Reproducible Plots with Matplotlib.html)

---

## Converting to a Jekyll Blog Post

Let's now take a look at a more complex example, in which we would like to convert a notebook into Markdown so that we can use it as the source for a blog post. For example, the "reproducible plots with matplotlib" notebook actually *is* the source for a blog post, which I created by using nbconvert to generate the relevant Markdown: http://www.jesshamrick.com/2016/04/13/reproducible-plots/

I found this functionality especially useful not just for blog posts, but for keeping "lab notebook" notes while I was doing my quals. For several of the papers that I read for my quals, I implemented the model in the paper in a notebook, and then published the notebook as part of a larger set of quals notes. For example: http://jhamrick.github.io/quals/probabilistic%20models%20of%20perception/2015/11/09/Weiss2002-ipynb.html

To convert to the specific Markdown format required by Jekyll, we need more sophisticated configuration options than what we can access on the command line. So, we'll create a configuration file for nbconvert:

In [ ]:
%%file nbconvert_config.py

"""
Based on: http://mcwitt.github.io/2015/04/29/jekyll_blogging_with_ipython3/
"""

from urllib.parse import quote
import os

def path2url(path):
    """Turn a file path into a URL"""
    parts = path.split(os.path.sep)
    return '{{ site.baseurl }}/notebooks/' + '/'.join(quote(part) for part in parts)

c = get_config()
c.NbConvertApp.export_format = 'markdown'
c.ExecutePreprocessor.enabled = True

# use the JekyllPreprocessor
c.Exporter.preprocessors = ['nbconvert_utils.JekyllPreprocessor']

# use the nbconvert-jekyll-post template file
c.MarkdownExporter.template_file = 'nbconvert-jekyll-post'
c.MarkdownExporter.filters = {'path2url': path2url}

First, this config file tells nbconvert to use the `JekyllPreprocessor` defined in the [nbconvert_utils.py](nbconvert_utils.py) file. This preprocessor parses out some metadata from the notebook, such as the title of the blog post and the date of publication.

Second, the config tells nbconvert to use a custom template, located in [nbconvert-jekyll-post.tpl](nbconvert-jekyll-post.tpl). This template, among other things, inserts the Jekyll blog post header (which includes the title, tags, etc.) and ensures that code will be highlighted with Python syntax.

To tell nbconvert to use this config file instead, we pass the `--config` option:

In [ ]:
%%bash

jupyter nbconvert --config=nbconvert_config.py "1 - Reproducible Plots with Matplotlib.ipynb"

Here are the results! [1 - Reproducible Plots with Matplotlib.md](1 - Reproducible Plots with Matplotlib.md)